In [7]:

import requests
from pprint import pprint
from api_list import ranking_api
from date_rounded import roundDate
from datetime import datetime
import time
import json

print(ranking_api)

https://api.nftgo.io/api/v1/ranking/collections?offset=0&limit=2&by=marketCap&interval=24h&asc=-1&rarity=-1&fields=marketCap,marketCapChange24h,relMarketCap,buyerNum24h,buyerNum24hChange24h,sellerNum24h,sellerNum24hChange24h,liquidity24h,liquidity24hChange24h,saleNum24h,saleNum24hChange24h,volume24h,relVolume24h,traderNum24h,traderNum24hChange24h,holderNum,holderNumChange24h,whaleNum,whaleNumChange24h,orderAvgPriceETH24h,orderAvgPriceETH24hChange24h,orderAvgPrice24h,orderAvgPrice24hChange24h,floorPrice,floorPriceChange24h


In [8]:


ranking_api = requests.get(ranking_api)
# /chart/marketcap-volume-V2?cid=61df623f786ee17194db7c7b&from=1628553600000&to=1636329600000
marketcap_volume_prefix = "https://api.nftgo.io/api/v1/collection-new/data/" 
# 6113f83c3126bb611b9b119d/chart/price-V2?cid=6113f83c3126bb611b9b119d&from=1646067600000&to=1647450000000&interval=24h&needGenDelta=0 
pricecap_prefix = "https://api.nftgo.io/api/v1/collection-new/data/"

total_nft = len(ranking_api.json()["data"]["list"])
print("LenData: ", total_nft)
print()
data = []
with open ('NFT_data.json', 'w') as f:
    for i in range(total_nft):
        name = ranking_api.json()["data"]["list"][i]["name"]
        id = ranking_api.json()["data"]["list"][i]["id"]
        slug = ranking_api.json()["data"]["list"][i]["slug"]
        tag = ranking_api.json()["data"]["list"][i]["tags"][0]
        listedTime = roundDate(ranking_api.json()["data"]["list"][i]["listedTime"])

        startTime = listedTime
        priceValueX = []
        priceValueY = []
        print(startTime)
        print(int(time.time())*1000)
        while startTime < (int(time.time())*1000):
            price_api = pricecap_prefix + str(id) + "/chart/price-V2?cid=" + str(id) + "&from=" + str(startTime) + "&to=" + str(startTime + 7776000000) + "&interval=24h&needGenDelta=0"
            price_api = requests.get(price_api)
            priceValueX = priceValueX + price_api.json()["data"]["avgPrice"]["values"]["x"]
            priceValueY = priceValueY + price_api.json()["data"]["avgPrice"]["values"]["y"]
            startTime += 7776000000
        # marketcap_volume_api = marketcap_volume_prefix + str(id) + "/chart/marketcap-volume-V2?cid=" + "&from=" + str(id) + str(1647423046) + "&to=" + str(1647423046) + "&range=all"
        # marketcap_volume_api = requests.get(marketcap_volume_api)
        # dayOneCap = marketcap_volume_api.json()["data"]["marketCap"]["values"]["y"][0]
                
        r = requests.get(f'https://nftgo.io/collection/{slug}/overview').text
        start = r.find('"nftNum"') + 9
        finish = r.find(',', start)
        amount = r[start:finish]

        date = datetime.utcfromtimestamp(listedTime//1000)
        date = date.strftime('%Y-%m')

        nftData = {'name': name, 'id': id, 'listedTimeUnix': listedTime, 'listedTimeDate': str(date), 'totalMinted': int(amount), 'tag': tag, 'avgprice': [{ "x": priceValueX, "y":priceValueY}]}
        data.append(nftData)
    json.dump(data, f)

LenData:  2

1628553600000
1647435984000
1628553600000
1647435992000


In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.DataFrame(data)
# df.sort_values(by=['listedTime'])

print(df)

plt.figure(figsize = (30,16))

p = sns.scatterplot(data=df, x="listedTimeDate", y="totalMinted", size="listedMarketCap")
p.set( yscale="log")
# sns.barplot(x="name", y="listedMarketCap", data=df)


                   name                        id  listedTimeUnix  \
0           CryptoPunks  6113f83a3126bb611b9b117c   1628553600000   
1  Bored Ape Yacht Club  6113f83c3126bb611b9b119d   1628553600000   

  listedTimeDate  totalMinted  tag  \
0        2021-08         9999  PFP   
1        2021-08        10000  PFP   

                                            avgprice  
0  [{'x': [1628553600000, 1628640000000, 16287264...  
1  [{'x': [1628553600000, 1628640000000, 16287264...  


ValueError: Could not interpret value `listedMarketCap` for parameter `size`

<Figure size 2160x1152 with 0 Axes>